## 賃料についてのドメイン知識


- https://lobotomy-project.org/marketpricerent/
- https://ameblo.jp/enjoytokyolovelivelife/entry-11084920720.html


## パッケージインポート

In [75]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats
from scipy.stats import norm, skew
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import matplotlib
font = {'family': 'Yu Mincho'}
matplotlib.rc('font', **font)

pd.set_option('max_columns',1000)
pd.set_option('max_rows',1000)

import warnings
warnings.filterwarnings('ignore')

import re
import geocoder
from math import sin, cos,radians
import math
import pyproj
from geopy.distance import great_circle, vincenty
from tqdm import tqdm

import os
import gc
print(os.listdir("../input"))
print(os.listdir("../submit"))

n_splits = 10
SEED=1234

['data', 'geo_data.csv', 'line.csv', 'location 保存用.txt', 'location.csv', 'location.txt', 'other', 'station 保存用.txt', 'station.csv', 'station.txt', 'test.csv', 'test.pkl', 'tokyo23', 'train.csv', 'train.pkl', '~$駅マンション平均単価.xlsx', '駅マンション平均単価.xlsx']
['feature.csv', 'sample_submit.csv', 'submit1.csv', 'submit10.csv', 'submit11.csv', 'submit12.csv', 'submit13.csv', 'submit14.csv', 'submit15.csv', 'submit16.csv', 'submit17.csv', 'submit18.csv', 'submit19.csv', 'submit2.csv', 'submit20.csv', 'submit21.csv', 'submit22.csv', 'submit3.csv', 'submit4.csv', 'submit5.csv', 'submit6.csv', 'submit7.csv', 'submit8.csv', 'submit9.csv', '特徴量について.xlsx']


In [76]:
def get_location_lonlat(df):
    '''
    緯度経度取得関数
    '''
    loc = pd.read_table('../input/location.txt',encoding='cp932',delimiter=',')
    loc = loc.rename(columns={'location':'所在地'})
    
    ## -の正規化
    df['所在地'] = df['所在地'].apply(lambda x: x.replace('−','－'))
    loc['所在地'] = loc['所在地'].apply(lambda x: x.replace('−','－'))
    
    ## lon, lat取得
    lat_map = loc.groupby('所在地')[' lat'].max()
    lon_map = loc.groupby('所在地')[' lon'].max()
    df['loc_lat'] = df['所在地'].map(lat_map)
    df['loc_lon'] = df['所在地'].map(lon_map)
    
    return df

In [77]:
def get_center_dis(df):
    '''
    国会議事堂からの距離を求める関数
    '''
    df = get_location_lonlat(df)
    df['center_dis'] = df.apply(lambda x: vincenty((x['loc_lat'],x['loc_lon']),(35.6759323, 139.7450316)).meters, axis=1)
    return df

In [78]:
def get_city(df):
    '''
    区を抽出
    '''
    ## 市町村をわける
    siku = ["千代田区","中央区","港区","新宿区","文京区","台東区","墨田区","江東区","品川区","目黒区","大田区","世田谷区","渋谷区","中野区",
            "杉並区","豊島区","北区","荒川区","板橋区","練馬区","足立区","葛飾区","江戸川区","八王子市","立川市","武蔵野市","三鷹市","青梅市",
            "府中市","昭島市","調布市","町田市","小金井市","小平市","日野市","東村山市","国分寺市","国立市","福生市","狛江市","東大和市",
            "清瀬市","東久留米市","武蔵村山市","多摩市","稲城市","羽村市","あきる野市","西東京市","瑞穂町","日の出町","檜原村","奥多摩町",
            "大島町","利島村","新島村","神津島村","三宅村","御蔵島村","八丈町","青ヶ島村","小笠原村",]
    
    df['city'] = df['所在地'].apply(lambda x: x.replace('東京都',''))
    df['city'] = df['city'].apply(lambda x: [s for s in siku if s in x][0])
    
    return df

In [79]:
def get_city2(df):
    '''
    街をいれる
    '''
    
    siku = ["千代田区","中央区","港区","新宿区","文京区","台東区","墨田区","江東区","品川区","目黒区","大田区","世田谷区","渋谷区","中野区",
            "杉並区","豊島区","北区","荒川区","板橋区","練馬区","足立区","葛飾区","江戸川区","八王子市","立川市","武蔵野市","三鷹市","青梅市",
            "府中市","昭島市","調布市","町田市","小金井市","小平市","日野市","東村山市","国分寺市","国立市","福生市","狛江市","東大和市",
            "清瀬市","東久留米市","武蔵村山市","多摩市","稲城市","羽村市","あきる野市","西東京市","瑞穂町","日の出町","檜原村","奥多摩町",
            "大島町","利島村","新島村","神津島村","三宅村","御蔵島村","八丈町","青ヶ島村","小笠原村",]
        
     ## 市区町村を抜く
    df['city2'] = df['所在地'].apply(lambda x: x.replace('東京都',''))
    for s in siku:
        df['city2'] = df['city2'].apply(lambda x: x.replace(s,''))   
        
    df['city2'] = df['city2'].apply(lambda x: re.split('\d+',x)[0])
    
    return df

In [80]:
def get_floor_plan(df):
    '''
    間取りを詳しく分けたものを取得
    '''
    df['S'] = df['間取り'].apply(lambda x: 1 if '納戸' in x else 0)
    
    df['間取り'] = df['間取り'].apply(lambda x: x.replace('(納戸)','').replace('+S',''))
    
    df['ishitsu'] = df['間取り'].apply(lambda x:int(re.sub('\\D', '', x)))
    df['L'] = df['間取り'].apply(lambda x:1 if 'L' in x else 0)
    df['D'] = df['間取り'].apply(lambda x:1 if 'D' in x else 0)
    df['K'] = df['間取り'].apply(lambda x:1 if 'K' in x else 0)
    
    return df

In [81]:
def get_age(df):
    '''
    築年数に関して、新築フラグと築年数を求める
    '''
    ## 新築フラグ
    df['new'] = df['築年数'].apply(lambda x:1 if x=='新築' else 0)
    
    ## 新築を0年0カ月に
    df['築年数'] = df['築年数'].apply(lambda x:'0年0ヶ月' if x=='新築' else x)
    
    ## 築年数を月計算
    df['築年数'] = df['築年数'].apply(lambda x: int(re.sub('\\D', '', x.split('年')[0]))*12 + int(re.sub('\\D', '', x.split('年')[1])))
    
    # 外れ値消去
    df = df[df['築年数']<6000]
    
    return df

In [82]:
def get_area(df):
    '''
    面積と畳計算したものを取得
    '''
    df['面積'] = df['面積'].apply(lambda x: x.rstrip('m2')).replace('','0').astype(float)
    
    ## 外れ値削除
    df = df[df['面積']<400]
    
    ## 畳計算
    df['畳'] = df['面積'].apply(lambda x: int(x/1.45))
    
    return df

In [83]:
def get_floor(df):
    '''
    戸建て・地下フラグ、所在階、総階数を入れる
    '''
    ## 欠損値を埋める
    df['所在階']= df['所在階'].fillna('0階')
    
    ## 総階数
    df['総階数'] = df['所在階'].apply(lambda x: str(x).split('／')[1] if len(str(x).split('／'))>1 else x)
    df['総階数'] = df['総階数'].apply(lambda x: str(x).split('階')[0]).astype('int')
    
    ## 戸建てと地下をダミー変数に
    df['戸建て'] = df['所在階'].apply(lambda x: 0 if '／' in str(x) else 1)
    df['地下あり'] = df['所在階'].apply(lambda x: int(re.sub('\\D', '', str(x).split('地下')[1].split('階')[0])) if '地下' in str(x) else 0)
    
    ## 所在階を数値に
    df['所在階'] = df['所在階'].apply(lambda x: str(x).split('／')[0] if len(str(x).split('／'))>1 else x)
    df['所在階'] = df['所在階'].apply(lambda x: '-'+ str(x).split('階')[0].replace('地下','') if '地下' in str(x).split('階')[0] 
                                     else str(x).split('階')[0])
    df['所在階'] = df['所在階'].replace('',0).astype(int)
    
    return df

In [84]:
def get_construct(df):
    '''
    契約期間が「まで」かどうか・借家のダミーと契約期間の月数を取得
    '''
    ## 欠損値を埋める
    df['契約期間'] = df['契約期間'].fillna('-')
    
    ## 借家フラグ
    df['借家'] = df['契約期間'].apply(lambda x: 1 if '定期借家' in x else 0)
    
    # までダミー
    df['契約期間(まで)'] = df['契約期間'].apply(lambda x: 1 if 'まで' in x else 0)
    
    ## 契約期間の月数
    df['契約期間'] = df['契約期間'].apply(lambda x:x.split()[0])
    df['契約期間'] = df['契約期間'].apply(lambda x: 12*int(x.split('年間')[0]) if '年間' in x
                                         else 12*int(x.split('年')[0])+int(re.sub('\\D', '', x.split('年')[1])) if ('月間' in x) and ('年' in x)
                                         else int(re.sub('\\D','',x)) if '月間' in x
                                         else 12*(int(re.sub('\\D', '', x.split('年')[0]))-2019)+(int(re.sub('\\D', '', x.split('年')[1]))-10)  
                                                 if ('まで' in x)and (int(re.sub('\\D', '', x.split('年')[1]))-10>=0)
                                         else 12*(int(re.sub('\\D', '', x.split('年')[0]))-2020)+13+(int(re.sub('\\D', '', x.split('年')[1]))-10)
                                                 if  'まで' in x
                                         else x)
    df['契約期間'] = df['契約期間'].apply(lambda x: int(x) if x!='-' else 0)
    
    return df

In [85]:
def get_BathToilet(df):
    '''
    バス・トイレの機能を全てダミー変数に
    '''
    
    ## 欠損値を埋める
    df['バス・トイレ'].fillna('-',inplace=True)
    
    ## 何が含まれているのか
    b_t = set()
    bath = df['バス・トイレ'].apply(lambda x:str(x).split('／')).values
    for lis in bath:
        for i in lis:
            i = i.replace('\t','')
            b_t.add(i)

    ## それぞれにフラグを立てる
    for bt in b_t:
        df[bt] = df['バス・トイレ'].apply(lambda x: 1 if bt in x else 0)
    
    return df

In [86]:
def get_kitchen(df):
    '''
    キッチン機能のフラグ
    '''
    ## 欠損値を埋める
    df['キッチン'].fillna('-',inplace=True)
    
    kit = set()
    kitchen = df['キッチン'].apply(lambda x:str(x).split('／')).values
    for lis in kitchen:
        for i in lis:
            i = i.replace('\t','')
            kit.add(i)

    ## それぞれにフラグを立てる
    fe = []
    for k in kit:
        if k=='-':
            continue
        df[k] = df['キッチン'].apply(lambda x: 1 if k in x else 0)
    
    return df

In [87]:
def get_broadcast(df):
    '''
    放送・通信のダミー取得
    '''
    ## 欠損値を埋める
    df['放送・通信'].fillna('-',inplace=True)
    
    ## 何が含まれているのか
    bro = set()
    broad = df['放送・通信'].apply(lambda x:str(x).split('／')).values
    for lis in broad:
        for i in lis:
            i = i.replace('\t','')
            bro.add(i)
        
    ## それぞれにフラグを立てる
    for br in bro:
        if br=='-':
            continue
        df[br] = df['放送・通信'].apply(lambda x: 1 if br in x else 0)
    
    return df

In [88]:
def get_env(df):
    '''
    周辺環境施設までの距離と個数
    '''
    ## 欠損値を埋める
    df['周辺環境'].fillna('-',inplace=True)
    
    ## 何が含まれているのか
    en = set()
    env = df['周辺環境'].apply(lambda x:str(x).split('\t')).values
    for lis in env:
        for i in lis:
            if i =='-':
                continue
            i = re.findall('【.*】',i)[0]
            en.add(i)
    
    for e in en:
        ## 周辺環境との最短距離
        df[str(e)+'_dis'] = df['周辺環境'].apply(lambda x: int(str(x).split()[str(x).split().index(e)+1].replace('m','')) if e in str(x).split() else 0)
        
        ## 周辺環境の個数
        df[str(e)+'_num'] = df['周辺環境'].apply(lambda x: str(x).split().count(e))
        
    return df

In [89]:
def get_facilities(df):
    '''
    室内設備のダミー
    '''
    
    ## 欠損値を埋める
    df['室内設備'].fillna('-',inplace=True)
    
    ## 何が含まれているのか
    fac = set()
    facility = df['室内設備'].apply(lambda x:str(x).split()).values
    for lis in facility:
        for i in lis:
            i = i.replace('／','')
            fac.add(i)

    ## それぞれにフラグを立てる
    fe = []
    for f in fac:
        if f =='-' or f=='':
            continue
        df[f] = df['室内設備'].apply(lambda x: 1 if f in x else 0)
    
    return df

In [90]:
def get_park(df):
    '''
    駐車場に関して
    '''
    ## 欠損値を埋める
    df['駐車場'].fillna('-',inplace=True)
    
    park = ['駐輪場','駐車場','バイク置き場']
    
    for p in park:
        ## 駐車場の状況を
        df[p] = df['駐車場'].apply(lambda x: x.split()[x.split().index(p)+1].split('(')[0] if p in x.split() else '-')
        
    return df

In [91]:
def get_access(df):
    '''
    最寄り駅までのアクセスに関して
    '''
    
    ## 最寄り駅を線、駅名、かかる時間に分ける
    df['路線'] = df['アクセス'].apply(lambda x: x.split()[0])
    df['駅'] = df['アクセス'].apply(lambda x: x.split()[1])
    df['最寄り駅まで'] = df['アクセス'].apply(lambda x: x.split()[2].replace('/',''))
    df['最寄り駅まで2'] = df['アクセス'].apply(lambda x: x.split()[3] if len(x.split())>=4 else '')
    df['最寄り駅まで2'] = df['最寄り駅まで2'].apply(lambda x: x if re.search("[0-9]",x) else  '')
    df['最寄り駅まで'] = df['最寄り駅まで'] + df['最寄り駅まで2']
    df.drop('最寄り駅まで2',axis=1, inplace=True)
    
    ## バス車を使うか
    df['バス・車'] = df['最寄り駅まで'].apply(lambda x: 1 if ('バス' in x) or ('車' in x) else 0)

    return df

In [92]:
def get_station_latlon(df):
    '''
    駅の緯度経度を取得
    '''
    
    df = get_access(df)
    
    ## 駅の緯度経度
    sta = pd.read_table('../input/station.txt',encoding='cp932',delimiter=',')
    sta = sta.rename(columns={'station':'駅'})
    
    ## 最寄り駅の緯度経度追加
    ## lon, latのマッピング
    lat_map = sta.groupby('駅')['lat'].max()
    lon_map = sta.groupby('駅')['lon'].max()
    df['sta_lat'] = df['駅'].map(lat_map)
    df['sta_lon'] = df['駅'].map(lon_map)
    
    return df

In [93]:
def get_station_dis(df):
    '''
    駅との距離を求める
    '''
    df = get_location_lonlat(df)
    df = get_station_latlon(df)
    ## 最寄り駅との距離
    df['station_dis'] = df.apply(lambda x: vincenty((x['loc_lat'],x['loc_lon']),(x['sta_lat'],x['sta_lon'])).meters, axis=1)
    
    return df

In [94]:
def get_23info(df):
    '''
    23区の情報を取得
    '''
    ## 12 区の情報

    df1 = pd.read_csv('../input/tokyo23/area.csv',encoding='utf-8')
    df2 = pd.read_csv('../input/tokyo23/daytime_popu.csv',encoding='utf-8')
    df3 = pd.read_csv('../input/tokyo23/dwelling.csv',encoding='utf-8')
    df4 = pd.read_csv('../input/tokyo23/households.csv',encoding='utf-8')
    df5 = pd.read_csv('../input/tokyo23/income.csv',encoding='utf-8')
    df6 = pd.read_csv('../input/tokyo23/move_popu.csv',encoding='utf-8')
    df7 = pd.read_csv('../input/tokyo23/office.csv',encoding='utf-8')
    df8 = pd.read_csv('../input/tokyo23/population.csv',encoding='utf-8')
    df9 = pd.read_csv('../input/tokyo23/popu_tran.csv',encoding='shift-jis')
    df10 = pd.read_csv('../input/tokyo23/revenue.csv',encoding='utf-8')
    df11 = pd.read_csv('../input/tokyo23/households_num.csv',encoding='utf-8')

    data = pd.merge(df1,df2,on='地域',how='outer')
    data = pd.merge(data,df3,on='地域',how='outer')
    data = pd.merge(data,df4,on='地域',how='outer')
    data = pd.merge(data,df5,on='地域',how='outer')
    data = pd.merge(data,df6,on='地域',how='outer')
    data = pd.merge(data,df7,on='地域',how='outer')
    data = pd.merge(data,df8,on='地域',how='outer')
    data = pd.merge(data,df9,on='地域',how='outer')
    data = pd.merge(data,df10,on='地域',how='outer')
    data = pd.merge(data,df11,on='地域',how='outer')

    data = data.rename(columns={'地域':'city'})
    data = data.query('city!="区部"')

    df = get_city(df)

    for f in data.columns:
        if f =='city':
            continue
        city_map = data.groupby('city')[f].mean().to_dict()
        df[f] = df.city.map(city_map)
    
    return df

In [95]:
def get_stove(df):
    '''
    コンロの数を数える
    '''
    df = get_kitchen(df)
    
    ## コンロ数は数値に
    df['コンロ'] = df['コンロ1口'] + df['コンロ2口']*2 + df['コンロ3口']*3 + df['コンロ4口以上']*4+df['IHコンロ']+df['ガスコンロ']

    ## 工数不明は最頻値の2で埋める
    df['コンロ設置可'] = df['コンロ設置可（コンロ1口）'] + df['コンロ設置可（コンロ2口）']*2 + df['コンロ設置可（コンロ3口）']*3 + df['コンロ設置可（コンロ4口以上）']*4 + df['コンロ設置可（口数不明）']*2
    ## 合計
    df['コンロ計'] = df['コンロ'] + df['コンロ設置可']
    
    return df

### 全前処理

In [96]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
submit = pd.read_csv('../submit/sample_submit.csv',header=None)
submit = submit.rename(columns={0:'id',1:'target'})
df = pd.concat([train, test])

In [97]:
%%time
df = get_23info(df)
df = get_BathToilet(df)
df = get_access(df)
df = get_age(df)
df = get_area(df)
df = get_broadcast(df)
df = get_center_dis(df)
df = get_city(df)
df = get_city2(df)
df = get_env(df)
df = get_facilities(df)
df = get_floor(df)
df = get_floor_plan(df)
df = get_kitchen(df)
df = get_location_lonlat(df)
df = get_park(df)
df = get_station_dis(df)
df = get_station_latlon(df)
df = get_stove(df)

Wall time: 53.4 s


In [98]:
df.head(1)

,id,アクセス,キッチン,バス・トイレ,周辺環境,契約期間,室内設備,建物構造,所在地,所在階,放送・通信,方角,築年数,賃料,間取り,面積,駐車場,city,総面積,宅地,その他,公園等,未利用 地等,道路等,農用地,水 面/河 川/水 路,森 林,原 野,昼間人口,常住人口,流入人口,流出人口,昼間人口密度,昼夜間人口比率,~29,30 ～49,50 ～69,70 ～99,100 ～149,150~,住居総数,持ち家,借家,1人,2人,3人,4人,5人以上,1世帯当たり人員,年収,2017_増減,2017_転入,2017_転出,2016_増減,2016_転入,2016_転出,2015_増減,2015_転入,2015_転出,2014_増減,2014_転入,2014_転出,2013_増減,2013_転入,2013_転出,事業所数,従業者数,2 013,2 014,2 015,2 016,2 017,1980年,1985年,1990年,1995年,2000年,2005年,2010年,2015年,2020年,2025年,2030年,2035年,2040年,2045年,特別区民税,特別区交付金,使用料手数料,国庫支出金,都支出金,歳入合計,総世帯数,共同トイレ,温水洗浄便座,バスなし,専用バス,トイレなし,共同バス,専用トイレ,洗面台独立,シャワー,-,脱衣所,追焚機能,バス・トイレ別,浴室乾燥機,路線,駅,最寄り駅まで,バス・車,new,畳,有線放送,BSアンテナ,CSアンテナ,CATV,インターネット対応,高速インターネット,光ファイバー,インターネット使用料無料,loc_lat,loc_lon,center_dis,city2,【学校】_dis,【学校】_num,【ドラッグストア】_dis,【ドラッグストア】_num,【公園】_dis,【公園】_num,【郵便局】_dis,【郵便局】_num,【幼稚園・保育園】_dis,【幼稚園・保育園】_num,【レンタルビデオ】_dis,【レンタルビデオ】_num,【コインパーキング】_dis,【コインパーキング】_num,【飲食店】_dis,【飲食店】_num,【クリーニング】_dis,【クリーニング】_num,【図書館】_dis,【図書館】_num,【月極駐車場】_dis,【月極駐車場】_num,【コンビニ】_dis,【コンビニ】_num,【総合病院】_dis,【総合病院】_num,【銀行】_dis,【銀行】_num,【デパート】_dis,【デパート】_num,【病院】_dis,【病院】_num,【大学】_dis,【大学】_num,【小学校】_dis,【小学校】_num,【スーパー】_dis,【スーパー】_num,都市ガス,地下室,水道その他,3面採光,石油暖房,ルーフバルコニー,洗濯機置場なし,エアコン付,汲み取り,タイル張り,室外洗濯機置場,バリアフリー,フローリング,ペアガラス,シューズボックス,クッションフロア,井戸,ロフト付き,オール電化,冷房,二重サッシ,2面採光,出窓,床下収納,トランクルーム,プロパンガス,二世帯住宅,床暖房,ガスその他,バルコニー,エレベーター,専用庭,ガス暖房,浄化槽,排水その他,敷地内ごみ置き場,公営水道,ウォークインクローゼット,24時間換気システム,防音室,室内洗濯機置場,下水,総階数,戸建て,地下あり,S,ishitsu,L,D,K,コンロ設置可（コンロ2口）,コンロ設置可（コンロ3口）,ガスコンロ,コンロ3口,給湯,コンロ2口,電気コンロ,カウンターキッチン,コンロ設置可（口数不明）,L字キッチン,コンロ設置可（コンロ1口）,冷蔵庫あり,コンロ設置可（コンロ4口以上）,IHコンロ,コンロ4口以上,システムキッチン,独立キッチン,コンロ1口,駐輪場,バイク置き場,sta_lat,sta_lon,station_dis,コンロ,コンロ設置可,コンロ計
0,1,都営三田線\t西巣鴨駅\t徒歩4分\t\t埼京線\t板橋駅\t徒歩14分\t\t都電荒川線\...,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄便座,【小学校】 495m\t【大学】 461m\t【小学校】 962m\t【公園】 1103m\...,2年間,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,RC（鉄筋コンクリート）,東京都北区滝野川３丁目,1,インターネット対応／\tCATV／\tCSアンテナ／\tBSアンテナ,南東,117,75000.0,1K,20.01,-,北区,2053.2,1166.5,69.9,171.3,48.5,455.1,0.8,116.5,3.3,21.4,329753,341076,84063,95386,16000,97,37970.0,44190.0,35010.0,27040.0,12130.0,6770.0,163110.0,67820.0,95290.0,90061.0,43596.0,24381.0,15387.0,4752.0,1.89,369.0,1075,13874,12799,1443,13739,12296,1407,13358,11951,1735,13139,11404,1234,12393,11159,12536,124765,335273,338090,341076,345065,348425,387458.0,367579.0,354647.0,334127.0,326764.0,330412.0,335544.0,341076.0,343622.0,344551.0,344086.0,342553.0,339942.0,336281.0,273.0,497.0,29.0,265.0,96.0,1454.0,190156.0,0,1,0,1,0,0,1,0,1,0,0,0,1,1,都営三田線,西巣鴨駅,徒歩4分,0,0,13,0,1,1,1,1,0,0,0,35.748456,139.7288,8179.773244,滝野川,0,0,956,1,1103,1,1246,1,0,0,0,0,0,0,378,1,0,0,0,0,0,0,588,1,0,0,0,0,0,0,0,0,461,1,495,2,311,2,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,1,1,12,0,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,空有,-,35.743694,139.728559,528.813124,3,0,3


In [99]:
## pickle化
#train.to_pickle('../input/train.pkl')
#test.to_pickle('../input/test.pkl')